In [5]:
import pandas as pd

#FILE PATHS
KAGGLE_DATASET_FOLDER='./kaggle_data'

#Restaurants
ACCEPTS=KAGGLE_DATASET_FOLDER+'/chefmozaccepts.csv'
CUISINE=KAGGLE_DATASET_FOLDER+'/chefmozcuisine.csv'
HOURS=KAGGLE_DATASET_FOLDER+'/chefmozhours4.csv'
PARKING=KAGGLE_DATASET_FOLDER+'/chefmozparking.csv'
PLACES=KAGGLE_DATASET_FOLDER+'/geoplaces2.csv'

#Consumers
USER_CUISINE=KAGGLE_DATASET_FOLDER+'/usercuisine.csv'
USER_PAYMENT=KAGGLE_DATASET_FOLDER+'/userpayment.csv'
USER_PROFILE=KAGGLE_DATASET_FOLDER+'/userprofile.csv'

#User-Item-Rating
RATINGS=KAGGLE_DATASET_FOLDER+'/rating_final.csv'


In [7]:
#IMPORTING DATASET

#Restaurants
res_accepts_df=pd.read_csv(ACCEPTS)
res_cuisine_df=pd.read_csv(CUISINE)
res_hours_df=pd.read_csv(HOURS)
res_parking_df=pd.read_csv(PARKING)
restaurants_df=pd.read_csv(PLACES)

#Consumers
user_cuisine_df=pd.read_csv(USER_CUISINE)
user_payment_df=pd.read_csv(USER_PAYMENT)
user_profile_df=pd.read_csv(USER_PROFILE)

#User-Item-Rating
ratings_df=pd.read_csv(RATINGS) 
